In [ ]:
!pip install colab-xterm
%load_ext colabxterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 8.0 MB/s eta 0:00:00


Running one-shot and few-shot for mistral:7b.

In [ ]:
!ollama pull llama3.1:8b

In [ ]:
!curl http://127.0.0.1:11434

Ollama is running

In [ ]:
!pip3 install ollama ollama_llm

In [ ]:
# ---- Drop-in replacement if ollama_llm is missing ----
# Do NOT change logic. This wrapper matches your expected signature exactly.
# ---- Drop-in replacement: now supports THINKING ----
# import requests

# def run_ollama_inference_prompt(
#         api_url,
#         system_template,
#         model_name,
#         prompt,
#         jsonFormat=True,
#         temperature=0,
#         think=True,   # <--- Added
#         think_level=None  # for GPT-OSS: "low", "medium", "high"
#     ):

#     messages = [
#         {"role": "system", "content": system_template},
#         {"role": "user", "content": prompt},
#     ]

#     # Build payload
#     payload = {
#         "model": model_name,
#         "messages": messages,
#         "temperature": temperature,
#         "stream": False  # no streaming → no line-splitting errors
#     }


#     payload["think"] = think_level

#     if jsonFormat:
#         payload["format"] = "json"

#     # Send request
#     resp = requests.post(api_url, json=payload)
#     resp.raise_for_status()
#     data = resp.json()

#     # Extract thinking + answer
#     thinking = None
#     content = None

#     if isinstance(data, dict) and "message" in data:
#         thinking = data["message"].get("thinking")
#         content = data["message"].get("content")

#     # Return BOTH as a dict — safe for your existing loop
#     return {
#         "thinking": thinking,
#         "content": content
#     }


In [ ]:
# ---- Drop-in replacement if ollama_llm is missing ----
# Do NOT change logic. This wrapper matches your expected signature exactly.
import requests

def run_ollama_inference_prompt(api_url, system_template, model_name, prompt, jsonFormat=True, temperature=0):
    messages = [
        {"role": "system", "content": system_template},
        {"role": "user", "content": prompt},
    ]

    payload = {
        "model": model_name,
        "messages": messages,
        "temperature": temperature,
        "stream": False  # <- THIS FIXES YOUR ISSUE
    }

    if jsonFormat:
        payload["format"] = "json"

    resp = requests.post(api_url, json=payload)
    resp.raise_for_status()

    data = resp.json()

    # Standard Ollama response format
    if isinstance(data, dict) and "message" in data:
        return data["message"]["content"]

    return data

In [ ]:
import json
import requests
import tqdm
# from ollama_llm import run_ollama_inference_prompt
import pandas as pd
import asyncio
import os

ZEROSHOT_PROMPT = """
You are a careful classification model for short YouTube comments about a specific road accident case in India. Your only job is to map each comment to exactly ONE of 7 predefined labels. This is for a research project, so accuracy is critical.

Your job is to think like a human who might classify these comments based on the sentiment, incident details, emotions, judicial accountability and law and the context to the incident and not just explicitly at the face value of the comment in plain language.

Your task is to assign each comment to EXACTLY ONE of 7 labels.

The comment may be in English, Hindi, Marathi, Hinglish, or code-mixed, and may contain emojis.

think logically but also in context of the comment and feelings of the person writing the comments.

You must also take in account the context provided in this prompt for each comment and see what fits into the perfect category. the context here is for your refernece to accruactely classify the comments.

You must assign it to EXACTLY ONE of these 7 labels:
z
1 - Judicial Accountability and Policy Demands
    Comments focused on courts, judges, legal outcomes, punishment severity, or demands for stricter laws / legal action.

2 - Public Safety
    Concerns about road safety, traffic rules, reckless driving, drunk driving, minors driving, or how to prevent similar accidents.

3 - Socioeconomic Privilege
    Comments about rich vs poor, power, influence, money buying the law, unequal treatment in the justice system.

4 - Victim Sympathy
    Condolences, prayers, or emotional support for victims and their families (e.g., RIP, justice for X and Y, feeling sad for them).

5 - Anger or Outrage
    Strong negative emotions or frustration about the incident or its handling (towards the driver, system, society, etc.), but NOT mainly about:
    - courts / legal decisions (then use 1), or
    - rich/poor privilege (then use 3).

6 - Irrelevant/General Comments
    - Only emojis or mostly emojis
    - Jokes, memes, or one-word reactions with no clear link to the case
    - Off-topic comments
    - Comments too vague/short to clearly map to 1–5 or 7
    When in doubt about relevance, choose 6.

7 - Views on Similar Cases in the Past
    Mentions or questions about other past cases or comparisons (e.g., similar incidents, previous court outcomes).

DECISION RULES:
- Pick the SINGLE most central theme of the comment.
- If it is unclear or mostly irrelevant, choose 6.
- Do NOT output anything except ONE integer from 1 to 7.

Make sure the output only has that one number for each comment. If there are multiple sentences repeating the same thing in the same text, label only 1 number.

Now, considering all of the above, output ONLY the label number for the given comment in this JSON format:

{
    "label": <label_number>
}

Do not add any explanations or additional text.


The context for the comments is based on the incident that happened in india:

This summary, written as context for a Large Language Model (LLM), provides a detailed overview of the key events, aftermath, and reactions related to the 2024 Pune Porsche car crash in India, with an emphasis on the legal status of the adult accused in the cover-up.

The Incident and Initial Controversy
On 19 May 2024, in Pune's Kalyani Nagar, a 17-year-old minor, Vedant Agarwal, drove an unregistered Porsche Taycan Turbo S at an estimated speed of 200 km/h (124 mph), resulting in the deaths of two 24-year-old IT workers, Aneesh Awadhiya and Ashwini Koshta. The minor had consumed alcohol at two establishments before the crash. The immediate aftermath was marked by allegations of political influence from an elected member of the Maharashtra Legislative Assembly, Sunil Tingre, and a critical eight-hour delay in collecting the minor's blood sample.

The Juvenile Justice Board (JJB) initially granted the minor bail within hours, requiring him to write a 300-word essay on road accidents and undergo counseling. This lenient decision sparked massive national outrage, leading the JJB to later cancel the bail and remand him to an observation home. However, the Bombay High Court later ordered his release, terming the custody illegal, but mandated continued psychological counseling.

The Cover-up Conspiracy and Adult Legal Proceedings
The police investigation subsequently uncovered a concerted conspiracy to manipulate evidence, leading to the arrest of multiple adults:

Minor's Father (Vishal Agarwal): Arrested for giving his son the car and for endangering his life (under the Juvenile Justice Act), and for his role in the evidence tampering. He was granted bail in the case related to the Juvenile Justice Act and Motor Vehicles Act in June, but remains in custody in the blood sample swapping case.

Minor's Grandfather (Surendra Agarwal): Arrested for the wrongful confinement of the family's driver, allegedly coercing him to take the blame for the crash. He was granted bail in this specific confinement case in July 2024.

Minor's Mother: Arrested for providing her blood sample to be swapped with her son’s at Sassoon Hospital to show he was not drunk. She is currently out on bail.

Hospital Staff: Two senior doctors and a hospital staffer from Sassoon Hospital were arrested for their direct involvement in replacing the minor's blood samples.

Other Accused: A labour contractor and two others were also arrested on charges of conspiracy and destruction of evidence, including allegedly attempting to swap the blood samples of the minor's friends who were in the car.

The police have filed a 900-page chargesheet against seven accused for manipulating evidence. The sessions court is currently hearing arguments to frame charges against ten accused in the blood sample swapping and conspiracy case. The prosecution has argued that some accused are using delaying tactics.

Separately, the Pune Police appealed to the Juvenile Justice Board to try the minor as an adult, but the JJB dismissed this request in July 2025, citing a Supreme Court judgment that classified the offense as "serious," not "heinous," which is required for an adult trial. The police have since filed an appeal against the JJB's decision with the sessions court.

Public and Political Reactions
The leniency shown to the minor, contrasted with the treatment of the rich and poor, led to national outrage and sharp criticism from leaders like Rahul Gandhi, who highlighted the class disparity, stating the rich were asked to write an essay while ordinary drivers face prison.

"""

FEWSHOT_PROMPT = """
You are a careful classification model for short YouTube comments about a specific road accident case in India. Your only job is to map each comment to exactly ONE of 7 predefined labels. This is for a research project, so accuracy and precision are absolutely critical.

Your job is to think like a human who might classify these comments based on the sentiment, incident details, emotions, judicial accountability and law and the context to the incident and not just explicitly at the face value of the comment in plain language.

Your task is to assign each comment to EXACTLY ONE of 7 labels.

The comment may be in English, Hindi, Marathi, Hinglish, or code-mixed, and may contain emojis.

think logically but also in context of the comment and feelings of the person writing the comments.

You must also take in account the context provided in this prompt for each comment and see what fits into the perfect category. the context here is for your refernece to accruactely classify the comments.

You must assign it to EXACTLY ONE of these 7 labels:

1 - Judicial Accountability and Policy Demands
    Comments focused on courts, judges, legal outcomes, punishment severity, or demands for stricter laws / legal action.

2 - Public Safety
    Concerns about road safety, traffic rules, reckless driving, drunk driving, minors driving, or how to prevent similar accidents.

3 - Socioeconomic Privilege
    Comments about rich vs poor, power, influence, money buying the law, unequal treatment in the justice system.

4 - Victim Sympathy
    Condolences, prayers, or emotional support for victims and their families (e.g., “RIP”, “justice for X and Y”, “feel sad for them”).

5 - Anger or Outrage
    Strong negative emotions or frustration about the incident or its handling (towards the driver, system, society, etc.), but NOT mainly about:
    - courts / legal decisions (then use 1), or
    - rich/poor privilege (then use 3).

6 - Irrelevant/General Comments
    - Only emojis or mostly emojis
    - Jokes, memes, or one-word reactions with no clear link to the case
    - Off-topic comments
    - Comments too vague/short to clearly map to 1–5 or 7
    When in doubt about relevance, choose 6.

7 - Views on Similar Cases in the Past
    Mentions or questions about other past cases or comparisons (e.g., “like the Cameron Herrin case”).

FEW-SHOT EXAMPLES (for guidance):

Input: "The court must & should give capital punishment then only things will come to order"
Output: 1

Input: ":fire::victory_hand::OK_hand::hundred_points::crossed_fingers::thumbs_up:"
Output: 6

Input: ":red_heart::smiling_face:"
Output: 6

Input: "#justicefortheemplyee"
Output: 6

(because even though it mentions justice, it's just a hashtag and hence irrelevant to the topic of classification or context)

Input: "Does anyone know \"Cameron Herrin\" case ?????"
Output: 7

Input: "Money can buy law and everything"
Output: 3

Input: "Money can buy law and everything"
Output: 2

Input: "According to law under 18 can drink, murder ."
Output: 5

(you might think because of the word law, it should be 1, but think about the anger and outrage factor a person would face when they wrote this comment)

Input: "Idiot news reporters revealing Witness face"
Ouput: 6

(even though it sounds that it should be 5 cause of anger, it does not relate to the topic of discussion.)

Input: "Justice for Ashwini and Aneesh"
Output: 4

Input: "Judiciary,,,,,,moye moye ho gya"
Output: 1

DECISION RULES:
- Pick the SINGLE most central theme of the comment.
- If it is unclear or mostly irrelevant, choose 6.
- Do NOT output anything except ONE integer from 1 to 7.

Make sure the output only has that one number for each comment. If there are multiple sentences repeating the same thing in the same text, label only 1 number.

Now, considering all of the above, output ONLY the label number for the given comment.

Do NOT add any other characters, spaces, or explanation. Output the label number in json format:
{
    "label": <label_number>
}

Do not add any explanations or additional text.

The context for the comments is based on the incident that happened in india:

This summary, written as context for a Large Language Model (LLM), provides a detailed overview of the key events, aftermath, and reactions related to the 2024 Pune Porsche car crash in India, with an emphasis on the legal status of the adult accused in the cover-up.

The Incident and Initial Controversy
On 19 May 2024, in Pune's Kalyani Nagar, a 17-year-old minor, Vedant Agarwal, drove an unregistered Porsche Taycan Turbo S at an estimated speed of 200 km/h (124 mph), resulting in the deaths of two 24-year-old IT workers, Aneesh Awadhiya and Ashwini Koshta. The minor had consumed alcohol at two establishments before the crash. The immediate aftermath was marked by allegations of political influence from an elected member of the Maharashtra Legislative Assembly, Sunil Tingre, and a critical eight-hour delay in collecting the minor's blood sample.

The Juvenile Justice Board (JJB) initially granted the minor bail within hours, requiring him to write a 300-word essay on road accidents and undergo counseling. This lenient decision sparked massive national outrage, leading the JJB to later cancel the bail and remand him to an observation home. However, the Bombay High Court later ordered his release, terming the custody illegal, but mandated continued psychological counseling.

The Cover-up Conspiracy and Adult Legal Proceedings
The police investigation subsequently uncovered a concerted conspiracy to manipulate evidence, leading to the arrest of multiple adults:

Minor's Father (Vishal Agarwal): Arrested for giving his son the car and for endangering his life (under the Juvenile Justice Act), and for his role in the evidence tampering. He was granted bail in the case related to the Juvenile Justice Act and Motor Vehicles Act in June, but remains in custody in the blood sample swapping case.

Minor's Grandfather (Surendra Agarwal): Arrested for the wrongful confinement of the family's driver, allegedly coercing him to take the blame for the crash. He was granted bail in this specific confinement case in July 2024.

Minor's Mother: Arrested for providing her blood sample to be swapped with her son’s at Sassoon Hospital to show he was not drunk. She is currently out on bail.

Hospital Staff: Two senior doctors and a hospital staffer from Sassoon Hospital were arrested for their direct involvement in replacing the minor's blood samples.

Other Accused: A labour contractor and two others were also arrested on charges of conspiracy and destruction of evidence, including allegedly attempting to swap the blood samples of the minor's friends who were in the car.

The police have filed a 900-page chargesheet against seven accused for manipulating evidence. The sessions court is currently hearing arguments to frame charges against ten accused in the blood sample swapping and conspiracy case. The prosecution has argued that some accused are using delaying tactics.

Separately, the Pune Police appealed to the Juvenile Justice Board to try the minor as an adult, but the JJB dismissed this request in July 2025, citing a Supreme Court judgment that classified the offense as "serious," not "heinous," which is required for an adult trial. The police have since filed an appeal against the JJB's decision with the sessions court.

Public and Political Reactions
The leniency shown to the minor, contrasted with the treatment of the rich and poor, led to national outrage and sharp criticism from leaders like Rahul Gandhi, who highlighted the class disparity, stating the rich were asked to write an essay while ordinary drivers face prison.

"""

if __name__ == "__main__":
  models = ['llama3.1:8b']
  for model in models:
    for prompt in [ZEROSHOT_PROMPT, FEWSHOT_PROMPT]:
          df = pd.read_csv(f"/content/drive/MyDrive/2700_comments_final_labeled.csv")
          all_cases = df.to_dict(orient="records")
          if prompt == ZEROSHOT_PROMPT:
            pmt = "Zero-Shot"
          else:
            pmt = "Few-Shot"
          save_path = (
              f"/content/drive/MyDrive/tester/{pmt}/{model}_outputs.json"
          )
          os.makedirs(os.path.dirname(save_path), exist_ok=True)
          save_after = 1
          counter = 0
          outputs = []

          total_input_tokens = 0
          total_output_tokens = 0
          total_cached_input_tokens = 0
          # for i in tqdm.tqdm(range(0, 10)):
          for i in tqdm.tqdm(range(0, len(df))):
              try:
                  sample_order = df.iloc[i]["text"]
                  response = run_ollama_inference_prompt(
                      api_url="http://localhost:11434/api/chat",
                      system_template=prompt,
                      model_name=model,
                      prompt=sample_order,
                      jsonFormat=False if model == 'llama3.1:8b' else True,
                      temperature=0
                  )
                  output  = response
                  llm_output_knowledge = output

                  output_json = df.iloc[i].to_dict()
                  output_json["llm_output_llama"] = llm_output_knowledge
                  output_json["llm_model"] = model
                  outputs.append(output_json)
              except Exception as e:
                  print("error", e)
                  output_json = df.iloc[i].to_dict()
                  output_json["llm_output_llama"] = {}
                  output_json["llm_model"] = model
                  outputs.append(output_json)

              counter += 1
              if counter % save_after == 0:
                  print("saving")
                  with open(save_path, "w") as f:
                      json.dump(outputs, f, indent=4)


          with open(save_path, "w") as f:
              json.dump(outputs, f, indent=4)


  0%|          | 0/2707 [00:00<?, ?it/s]

saving


  0%|          | 2/2707 [00:04<1:24:10,  1.87s/it]

saving


  0%|          | 3/2707 [00:04<52:06,  1.16s/it]  

...

saving


100%|█████████▉| 2706/2707 [14:34<00:00,  3.00it/s]

saving


100%|██████████| 2707/2707 [14:34<00:00,  3.09it/s]

saving
